In [25]:
import numpy as np
import pandas as pd

import seaborn as sns


In [26]:
df = pd.read_excel('assets/IndianFoodDatase/IndianFoodDatasetXLS.xlsx')
df.drop(['Srno','URL'],axis=1,inplace=True)

In [27]:
def arrange_ingredients(input_str):
    # Create an empty dictionary to store the parsed data
    output_dict = {}

    # Split the input string into individual ingredients
    ingredients = input_str.split(",")


    # Iterate over the ingredients and extract the name, quantity, and unit
    for ingredient in ingredients:
        name_quantity_unit = ingredient.split(" - ")
        name = name_quantity_unit[0]
        quantity_unit = name_quantity_unit[1].split(" ")
        print(quantity_unit)
        quantity = int(quantity_unit[0])
        unit = quantity_unit[1]
        lang_hi = quantity_unit[2][1:-1] if len(quantity_unit) > 2 else ""

        output_dict[name] = {"quantity": quantity, "unit": unit, "lang_hi": lang_hi}


In [29]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import pandas as pd
from jupyter_dash import JupyterDash

# Initialize the Dash app
app = JupyterDash(__name__)

# Define the columns for the table
columns = [
    {'name': 'Recipe Name', 'id': 'RecipeName'},
    {'name': 'Translated Recipe Name', 'id': 'TranslatedRecipeName'},
    {'name': 'Ingredients', 'id': 'Ingredients'},
    {'name': 'Translated Ingredients', 'id': 'TranslatedIngredients'},
    {'name': 'Prep Time (mins)', 'id': 'PrepTimeInMins'},
    {'name': 'Cook Time (mins)', 'id': 'CookTimeInMins'},
    {'name': 'Total Time (mins)', 'id': 'TotalTimeInMins'},
    {'name': 'Servings', 'id': 'Servings'},
    {'name': 'Cuisine', 'id': 'Cuisine'},
    {'name': 'Course', 'id': 'Course'},
    {'name': 'Diet', 'id': 'Diet'},
    {'name': 'Instructions', 'id': 'Instructions'},
    {'name': 'Translated Instructions', 'id': 'TranslatedInstructions'}
]


# Create the app layout
app.layout = html.Div([
    html.H1('Recipe Search'),
    dcc.Input(
        id='search-input',
        type='text',
        placeholder='Search recipes...',
        debounce=True,
        style={'margin': '10px'}
    ),
    dash_table.DataTable(
        id='table',
        columns=columns,
        page_size=10,
        page_current=0,
        page_action='custom',
        sort_action='custom',
        sort_mode='multi',
        style_table={'overflowX': 'scroll'},
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'textAlign': 'left'
        },
        style_header={
            'fontWeight': 'bold'
        }
    )
])

# Define the search callback function
@app.callback(
    dash.dependencies.Output('table', 'data'),
    dash.dependencies.Input('search-input', 'value'),
    dash.dependencies.State('table', 'page_current'),
    dash.dependencies.State('table', 'page_size'),
    dash.dependencies.State('table', 'sort_by')
)
def update_table(search_value, page_current, page_size, sort_by):
    print(search_value, page_current, page_size, sort_by)
    # Filter the dataframe by the search value
    if search_value:
        print("if block",search_value)
        filtered_df = df.apply(lambda row: row.astype(str).str.contains(search_value, case=False).any(), axis=1)
        df_filtered = df[filtered_df]
    else:
        print("else,block")
        df_filtered = df
        print(df.shape)

    # Sort the filtered dataframe
    if sort_by:
        df_sorted = df_filtered.sort_values(
            [col['id'] for col in sort_by],
            ascending=[col['desc'] for col in sort_by],
            inplace=False
        )
    else:
        df_sorted = df_filtered

    # Paginate the sorted dataframe
    df_paginated = df_sorted.iloc[page_current*page_size:(page_current+ 1)*page_size].to_dict('records')
    return df_paginated

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8054)


Dash is running on http://127.0.0.1:8054/

Dash app running on http://127.0.0.1:8054/
None 0 10 None
else,block
(6871, 13)
None 0 10 None
else,block
(6871, 13)
haldi 0 10 None
if block haldi


In [41]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import pandas as pd
from jupyter_dash import JupyterDash
from dash.dependencies import Output,Input,State


# Initialize the Dash app
app = JupyterDash(__name__)

# Define the columns for the table
columns = [
    {'name': 'Recipe Name', 'id': 'RecipeName'},
    {'name': 'Translated Recipe Name', 'id': 'TranslatedRecipeName'},
    {'name': 'Ingredients', 'id': 'Ingredients'},
    {'name': 'Translated Ingredients', 'id': 'TranslatedIngredients'},
    {'name': 'Prep Time (mins)', 'id': 'PrepTimeInMins'},
    {'name': 'Cook Time (mins)', 'id': 'CookTimeInMins'},
    {'name': 'Total Time (mins)', 'id': 'TotalTimeInMins'},
    {'name': 'Servings', 'id': 'Servings'},
    {'name': 'Cuisine', 'id': 'Cuisine'},
    {'name': 'Course', 'id': 'Course'},
    {'name': 'Diet', 'id': 'Diet'},
    {'name': 'Instructions', 'id': 'Instructions'},
    {'name': 'Translated Instructions', 'id': 'TranslatedInstructions'},
    # Add Twisty and Details columns
    {'name': '', 'id': 'twisty', 'presentation': 'markdown'},
    {'name': 'Details', 'id': 'details'}
]


# Create the app layout
app.layout = html.Div([
    html.H1('Recipe Search'),
    dcc.Input(
        id='search-input',
        type='text',
        placeholder='Search recipes...',
        debounce=True,
        style={'margin': '10px'}
    ),
dash_table.DataTable(
        id='table',
        columns=columns,
        page_size=10,
        page_current=0,
        page_action='custom',
        sort_action='custom',
        sort_mode='multi',
        style_table={'overflowX': 'scroll'},
        style_cell={
            'whiteSpace': 'normal',
            'height': 'auto',
            'textAlign': 'left'
        },
        style_header={
            'fontWeight': 'bold'
        }
    )

])
# Define the search callback function
@app.callback(
    dash.dependencies.Output('table', 'data'),
    dash.dependencies.Input('search-input', 'value'),
    dash.dependencies.State('table', 'page_current'),
    dash.dependencies.State('table', 'page_size'),
    dash.dependencies.State('table', 'sort_by')
)
def update_table(search_value, page_current, page_size, sort_by):
    # Filter the dataframe by the search value
    if search_value:
        filtered_df = df.apply(lambda row: row.astype(str).str.contains(search_value, case=False).any(), axis=1)
        df_filtered = df[filtered_df]
    else:
        df_filtered = df

    # Sort the filtered dataframe
    if sort_by:
        df_sorted = df_filtered.sort_values(
            [col['id'] for col in sort_by],
            ascending=[col['desc'] for col in sort_by],
            inplace=False
        )
    else:
        df_sorted = df_filtered

    # Paginate the sorted dataframe
    df_paginated = df_sorted.iloc[page_current*page_size:(page_current+ 1)*page_size]

    # Convert the dataframe to a list of dictionaries, adding the "details" key for twisty content
    data = []
    for index, row in df_paginated.iterrows():
        data.append({
            'Recipe Name': row['RecipeName'],
            'Translated Recipe Name': row['TranslatedRecipeName'],
            'Ingredients': row['Ingredients'],
            'Translated Ingredients': row['TranslatedIngredients'],
            'Prep Time (mins)': row['PrepTimeInMins'],
            'Cook Time (mins)': row['CookTimeInMins'],
            'Total Time (mins)': row['TotalTimeInMins'],
            'Servings': row['Servings'],
            'Cuisine': row['Cuisine'],
            'Course': row['Course'],
            'Diet': row['Diet'],
            'Instructions': row['Instructions'],
            'Translated Instructions': row['TranslatedInstructions'],
            'details': html.Div([
                html.P(f"Recipe Name: {row['RecipeName']}"),
                html.P(f"Translated Recipe Name: {row['TranslatedRecipeName']}"),
                html.P(f"Ingredients: {row['Ingredients']}"),
                html.P(f"Translated Ingredients: {row['TranslatedIngredients']}"),
                html.P(f"Prep Time (mins): {row['PrepTimeInMins']}"),
                html.P(f"Cook Time (mins): {row['CookTimeInMins']}"),
                html.P(f"Total Time (mins): {row['TotalTimeInMins']}"),
                html.P(f"Servings: {row['Servings']}"),
                html.P(f"Cuisine: {row['Cuisine']}"),
                html.P(f"Course: {row['Course']}"),
                html.P(f"Diet: {row['Diet']}"),
                html.P(f"Instructions: {row['Instructions']}"),
                html.P(f"Translated Instructions: {row['TranslatedInstructions']}")
            ])
        })

    return data

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8056)

Dash is running on http://127.0.0.1:8056/

Dash app running on http://127.0.0.1:8056/
